In [17]:
import os
import pickle

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, confusion_matrix, fbeta_score

from ipywidgets import interact, interactive, interact_manual, FloatSlider, IntSlider, Layout
from IPython.display import Image
import 
print(ipywidgets.__version__)



NameError: name 'ipywidgets' is not defined

In [2]:
def load_data(filepath):
    
    df = pd.read_csv(filepath, index_col=0)
    df['road_type_dummy'] = pd.get_dummies(df['road_type'], drop_first=True)
    df['night_dummy'] = pd.get_dummies(df['day_night'], drop_first=True)

    X = df[['acceleration_score', 'braking_score', 'turning_score', 'weaving_score',
       'drifting_score', 'speeding_score', 'follow_score', 'speed_mph', 'road_type_dummy', 'night_dummy']]
    y = df['class_normal_bool']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test


In [3]:
def load_model(filepath):
    
    with open(filepath, "rb") as pfile:
        rf_final = pickle.load(pfile)
        
    return rf_final

In [4]:
X_train, X_test, y_train, y_test = load_data('UAH-DRIVESET-classification.csv')

In [5]:
rf_final = load_model('models/rf_final.pickle')

y_rf_final_pred = rf_final.predict(X_test)
y_rf_final_pred_prob = rf_final.predict_proba(X_test)[:, 1]

print('Test set scores:\n')
print(f'\tFbeta (beta=2):\t {fbeta_score(y_test, y_rf_final_pred, beta=2):.4f}')


Test set scores:

	Fbeta (beta=2):	 0.9925


In [6]:
road_dummies_dict = {'Highway':0, 'Secondary':1}
night_dummies_dict = {'Day':0, 'Night':1}

class_dict = {0: 'Normal', 1:'Abnormal'}


In [7]:
def new_prediction(acc, brake, turn, weave, drift, speeding, follow, speed, road_type='Highway', night='Day'):
    road_type_bool = road_dummies_dict[road_type]
    night_bool = night_dummies_dict[night]

    new_test = np.array([acc, brake, turn, weave, drift, speeding, follow, speed, road_type_bool, night_bool]).reshape(1, -1)
    y_pred = rf_final.predict(new_test)[0]
    y_pred_prob = rf_final.predict_proba(new_test)[:, 1][0]
    
    print(f'Predicted: {class_dict[y_pred]} driving')
    print(f'(Probability: {y_pred_prob:.2f})')
    display(Image(filename=f'figures/{class_dict[y_pred]}.png', width=225) )
    
    return


In [8]:
interactive(lambda Acceleration: new_prediction(Acceleration,100,100,100,100,100,100,45,'Highway','Day'), Acceleration=(0,100,1))


interactive(children=(IntSlider(value=50, description='Acceleration'), Output()), _dom_classes=('widget-intera…

In [9]:
im = interact_manual(new_prediction, acc=IntSlider(100,0,100,1), brake=IntSlider(100,0,100,1), turn=IntSlider(100,0,100,1),
         weave=IntSlider(100,0,100,1), drift=IntSlider(100,0,100,1), speeding=IntSlider(100,0,100,1),
         follow=IntSlider(100,0,100,1), speed=IntSlider(40,30,80,1),
         road_type=['Highway','Secondary'], night=['Day','Night']);

labels = ['Acceleration', 'Braking', 'Turning', 'Weaving', 'Drifting', 'Speeding', 'Following']
for i,lab in enumerate(labels):
    im.widget.children[i].description = f'{lab}'

im.widget.children[i+1].description = 'Speed (mph)'
im.widget.children[i+2].description = 'Road type'
im.widget.children[i+3].description = 'Lighting'


interactive(children=(IntSlider(value=100, description='acc'), IntSlider(value=100, description='brake'), IntS…